<a href="https://colab.research.google.com/github/joon2332/corab/blob/master/electric_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd

In [ ]:
# data loading
df = pd.read_excel('./Current.xls', index_col=0, header=0, encoding='cp949', sep=',', dtype=({'OCCR_DT':str, 
'CALC_CC_VAL':float, 'AA':float})

In [ ]:
df.shape

df.head()

In [ ]:
# change pd > numpy
arr_current = df.values

In [ ]:
arr_current.shape

In [ ]:
type(df), type(arr_current)

In [ ]:
len(arr_current)

In [ ]:
arr_current[0:8, :]

In [ ]:
arr_current[8:9, 3]

In [ ]:
def split_xy5(dataset, time_steps, y_colum):
  x, y = list(), list()

  for i in range(len(dataset):
    x_end_number = i + time_steps
    y_end_number = x_end_number + y_colum

    if y_end_number > len(dataset):
      break

    tmp_x = dataset[i:x_end_number, :]
    tmp_y = dataset[x_end_number:y_end_number, 0]

    x.append(tmp_x)
    y.append(tmp_y)

  return np.array(x), np.array(y)

In [ ]:
x, y = split_xy5(arr_current, 8, 1)

print("x y dataset were splited")

In [ ]:
x.shape, y.shape

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, 
                                                    random=1111)
print("train, test dataset were splited")

In [ ]:
x_train.shape, x_test.shape

In [ ]:
x_train = np.reshape(x_train, (x_train.shape[0], 
                               x_train.shape[1] * x_train.shape[2]))

x_test = np.reshape(x_test, (x_test.shape[0], 
                             x_test.shape[1] * x_test.shape[2]))

print("trained data : ", x_train.shape)
print("test data : ", x_test.shape)

In [ ]:
x_train.shape, x_test.shape

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(x_train)

x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

print("nomalizing result : ", x_train_scaled[0, :])

In [ ]:
x_train_scaled = np.shape(x_train_scaled, (x_train_scaled.shape[0], 8, 8))
x_test_scaled = np.shape(x_test_scaled, (x_test_scaled[0], 8, 8))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [ ]:
model = Sequential()
model.add(LSTM(64, input_shape=(8, 8)))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

In [ ]:
model.compile(;oss='mse', optimizer='adam', metrics=['mse'])

In [ ]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience=5)
model.fit(x_train_scaled, y_train, validation_split=0.2, verbose=1, batch_size=12,
          epochs=5, callbacks=[early_stopping])

In [ ]:
loss, mse = model.evaluate(x_test_scaled, y_test, batch_size=1)

print('loss : ', loss)
print('mse : ', mse)

In [ ]:
y_pred = model.predict(x_test_scaled)

print('trainning completed')

for i in range(100):
  print('real : ', y_test[i], '/ predicted : ', y_pred[i])